<a href="https://colab.research.google.com/github/krautgortna/NLP_Project_SS22/blob/main/NLP_FakePolice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Project 2022

## Get data

In [2]:
import pandas as pd
url = 'https://raw.githubusercontent.com/SakibShahriar95/ANTiVax/main/Labeled/VaxMisinfoData.csv'
df = pd.read_csv(url)
df.head()

,id,is_misinfo
0,1344795424855642112,0
1,1344794858133860353,0
2,1344794822691983360,0
3,1344794752819077123,1
4,1344792070507134977,0


In [25]:
#  !pip install twarc
#  !twarc configure

In [ ]:
from twarc import Twarc
from urllib.request import urlretrieve

tweet_ids_url = 'https://raw.githubusercontent.com/SakibShahriar95/ANTiVax/main/Labeled/ids.txt'
tweet_ids = urlretrieve(tweet_ids_url, './ids.txt') # it's a file like object and works just like a file

df = pd.read_csv('./ids.txt')
df.head()

consumer_key="2lNba8Nvu9f0i5YgfmQhfh3mP"
consumer_secret="TGVv9QJhtkjSqm7QyMeLfpZm3IU3xjrrMabyae42VxbkaT5t0g"
access_token="1600730532-ZCeeHr9h033QRlC95BLZrpz85TRFN4pZ0w7vGhY"
access_token_secret="eoiufziPLBr0eYgR12gAoAEiTQjlnTkZ1BHBLXoDuQbtC"
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

df = pd.DataFrame()
for tweet in t.hydrate(open('./ids.txt')):
  row = { 'id': tweet['id'], 'text': tweet['text'] }
  df.append(row, ignore_index = True)



In [30]:
url = 'https://raw.githubusercontent.com/krautgortna/NLP_Project_SS22/main/data/hydrated_tweets.csv'
df = pd.read_csv(url)
df.text

0        My 87 year old grandmother has been quarantini...
1        I’m so excited that with this vaccine I can se...
2        So excited that I was able to close out 2020 o...
3        Well, it happened. My daughter used my grandch...
4        I’m getting my COVID-19 vaccine, baby! I’m so ...
                               ...                        
12292    Ooh! Potential #CoronaVac by @Sinovac side-eff...
12293    This rocks, very excited to get the vaccine as...
12294    We're excited to announce a COVID-19 vaccinati...
12295    Dec 2020:\nMOH: Vaccines are here!\nFPs: Yay! ...
12296    Fauci to #unvaccinated Americans: “get over th...
Name: text, Length: 12297, dtype: object

## Preprocessing

## Train model

## Tuning

## Debugging